In [1]:
import itertools

from fastapi import FastAPI, Form, Request, Response, File, Depends, HTTPException, status
from fastapi.responses import RedirectResponse
from fastapi.staticfiles import StaticFiles
from fastapi.templating import Jinja2Templates
from fastapi.encoders import jsonable_encoder
from langchain_community.llms import CTransformers, LlamaCpp
from langchain.chains import QAGenerationChain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.docstore.document import Document
from langchain_community.document_loaders import PyPDFLoader
from langchain.prompts import PromptTemplate
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.chains.summarize import load_summarize_chain
from langchain.chains import RetrievalQA
import os 
import json
import time
import uvicorn
import aiofiles
from PyPDF2 import PdfReader
import csv
from functools import lru_cache

app = FastAPI()

app.mount("/static", StaticFiles(directory="static"), name="static")

templates = Jinja2Templates(directory="templates")

#@lru_cache(maxsize=4096)  # Adjust maxsize as needed
def load_llm():
    # Load the locally downloaded model here
    config = {
        'gpu_layers': 10000,
        #'model_type': "mistral",
        'max_new_tokens' : 2048,
        'context_length' : 4096,
        'temperature' : 0,
        }

    llm = CTransformers(
        #model = 'mistral-7b-instruct-v0.1.Q5_K_M.gguf',#"mistral-7b-instruct-v0.1.Q4_K_S.gguf",
        model = '/home/rps/projects/Question-Answer-Generation-App-using-Mistral-7B/NousResearch/gguf/nous-hermes-2-solar-10.7b.Q5_K_M.gguf',
        config = config
    )
    return llm

#######################################################################
def load_llm_llama():
    model_id = 'qwen1_5-14b-chat-q5_0.gguf'
    n_gpu_layers = -1  # The number of layers to put on the GPU. The rest will be on the CPU. If you don't know how many layers there are, you can use -1 to move all to GPU.
    n_batch = 2048  # Should be between 1 and n_ctx, consider the amount of VRAM in your GPU.
    # Make sure the model path is correct for your system!
    llm = LlamaCpp(
        model_path= model_id,
        n_gpu_layers=n_gpu_layers,
        n_batch=n_batch,
        # callback_manager=callback_manager,
        temperature=0,
        max_tokens=2048,
        context_length = 4096,
        f16_kv=True,  # MUST set to True, otherwise you will run into problem after a couple of calls
        #top_p=1,
        verbose=False,  # Verbose is required to pass to the callback manager
    )
    return llm
########################################################################
llm = load_llm()

def file_processing(file_path):

    # Load data from PDF
    loader = PyPDFLoader(file_path)
    data = loader.load()

    question_gen = ''

    for page in data:
        question_gen += page.page_content
        
    splitter_ques_gen = RecursiveCharacterTextSplitter(
        chunk_size = 1000,
        chunk_overlap = 100
    )

    chunks_ques_gen = splitter_ques_gen.split_text(question_gen)

    document_ques_gen = [Document(page_content=t) for t in chunks_ques_gen]

    splitter_ans_gen = RecursiveCharacterTextSplitter(
        chunk_size = 300,
        chunk_overlap = 30
    )


    document_answer_gen = splitter_ans_gen.split_documents(
        document_ques_gen
    )

    return document_ques_gen, document_answer_gen

def pipe_vector(file_path):

    document_ques_gen, document_answer_gen = file_processing(file_path)

    #llm_ques_gen_pipeline = load_llm()

    prompt_template = """
    You are an expert at creating questions based in medicine materials and documentation.
    Your goal is to prepare a very good material for their exam and  tests.
    You do this by asking questions about the text below:

    ------------
    {text}
    ------------

    Create questions that will prepare medicine students for their tests.
    Make sure not to lose any important information.

    QUESTIONS:
    """

    PROMPT_QUESTIONS = PromptTemplate(template=prompt_template, input_variables=["text"])

    refine_template = ("""
    You are an expert at creating practice questions based on medicine and books.
    Your goal is to help to prepare medicine students for a test.
    We have received some practice questions to a certain extent: {existing_answer}.
    We have the option to refine the existing questions or add new ones.
    (only if necessary) with some more context below.
    ------------
    {text}
    ------------

    Given the new context, refine the original questions in Portuguese.
    If the context is not helpful, please provide the original questions.
    QUESTIONS:
    """
    )

    REFINE_PROMPT_QUESTIONS = PromptTemplate(
        input_variables=["existing_answer", "text"],
        template=refine_template,
    )

    ques_gen_chain = load_summarize_chain(llm = llm, 
                                            chain_type = "refine", 
                                            verbose = True, 
                                            question_prompt=PROMPT_QUESTIONS, 
                                            refine_prompt=REFINE_PROMPT_QUESTIONS)

    #ques = ques_gen_chain.run(document_ques_gen)
    ques_l = [ ques_gen_chain.run( [q]) for q in document_ques_gen]
    ques = "\n".join(ques_l).split("\n")
    embeddings = HuggingFaceBgeEmbeddings(model_name= "intfloat/multilingual-e5-large-instruct")  #"sentence-transformers/all-mpnet-base-v2")

    vector_store = FAISS.from_documents(document_answer_gen, embeddings)
    return ques, vector_store

def pipe_gen(vector_store, ques):
    # llm_answer_gen = load_llm()
    #ques_list = ques
    filtered_ques_list = [element for element in ques if element.endswith('?') or element.endswith('.')]
    #filtered_ques_list = ques
    answer_generation_chain = RetrievalQA.from_chain_type(llm=llm, 
                                                chain_type="stuff", 
                                                retriever=vector_store.as_retriever())
    return answer_generation_chain, filtered_ques_list


def get_csv (file_path):
    # answer_generation_chain, ques_list = llm_pipeline(file_path)
    vector_store = pipe_vector(file_path)
    answer_generation_chain, ques_list = pipe_gen(vector_store)

    base_folder = 'static/output/'
    if not os.path.isdir(base_folder):
        os.mkdir(base_folder)
    output_file = base_folder+"QAx.csv"
    with open(output_file, "w", newline="", encoding="utf-8") as csvfile:
        csv_writer = csv.writer(csvfile)
        csv_writer.writerow(["Question", "Helpful Answer"])  # Writing the header row
        #answer0=[]
        for question in ques_list:
            print("Question: ", question)
            answer = answer_generation_chain.run(question)
            #answer0.append(answer)
            print("Answer: ", answer)
            print("--------------------------------------------------\n\n")

            # Save answer to CSV file
            csv_writer.writerow([question, answer])
    return output_file

In [2]:
file_path = '//home/rps/Downloads/REUMATOLOGIA.pdf'
#file_path = '/home/rps/Downloads/ansiedade_4_8.pdf'
que,ans = file_processing(file_path)
len(que),len(ans)



(132, 525)

In [3]:
que[:2]

[Document(page_content='Jéssica Nicolau - 1 REUMATOLOGIA  \n \n_________________________________________________________________________________  \n \nTÓPICOS  \n1. Lúpus Eritematoso Sistêmico  \n● Introdução  \n● Quadro clínico  \n○ Manifestações sistêmicas  \n○ Manifestações laboratoriais  \n● Diagnóstico  \n● Tratamento  \n● Gravidez e contracepção  \n● Síndrome do Anticorpo Antifosfolipídeo (SAF)  \n2. Esclerodermia  \n● Introdução  \n● Classificação  \n○ Forma localizada  \n○ Forma sistêmica  \n■ Cutânea difusa  \n■ Cutânea limitada - CREST  \n■ Visceral  \n● Quadro clínico  \n● Diagnóstico  \n● Tratamento  \n3. Síndrome de Sjögren  \n● Introdução  \n● Quadro clínico  \n● Exames laboratoriais  \n \n \nJéssica Nicolau - 2 ● Diagnóstico  \n● Tratamento  \n4. Vasculites  \n● Introdução  \n● Vasculites de grandes vasos  \n○ Arterite Temporal  \n○ Arterite de Takayasu  \n● Vasculites de médios vasos  \n○ Poliarterite Nodosa (PAN)  \n○ Doença de Kawasaki  \n● Vasculites de pequenos vaso

In [4]:
for i in que:
    print(i)

page_content='Jéssica Nicolau - 1 REUMATOLOGIA  \n \n_________________________________________________________________________________  \n \nTÓPICOS  \n1. Lúpus Eritematoso Sistêmico  \n● Introdução  \n● Quadro clínico  \n○ Manifestações sistêmicas  \n○ Manifestações laboratoriais  \n● Diagnóstico  \n● Tratamento  \n● Gravidez e contracepção  \n● Síndrome do Anticorpo Antifosfolipídeo (SAF)  \n2. Esclerodermia  \n● Introdução  \n● Classificação  \n○ Forma localizada  \n○ Forma sistêmica  \n■ Cutânea difusa  \n■ Cutânea limitada - CREST  \n■ Visceral  \n● Quadro clínico  \n● Diagnóstico  \n● Tratamento  \n3. Síndrome de Sjögren  \n● Introdução  \n● Quadro clínico  \n● Exames laboratoriais  \n \n \nJéssica Nicolau - 2 ● Diagnóstico  \n● Tratamento  \n4. Vasculites  \n● Introdução  \n● Vasculites de grandes vasos  \n○ Arterite Temporal  \n○ Arterite de Takayasu  \n● Vasculites de médios vasos  \n○ Poliarterite Nodosa (PAN)  \n○ Doença de Kawasaki  \n● Vasculites de pequenos vasos associad

In [5]:
ans

[Document(page_content='Jéssica Nicolau - 1 REUMATOLOGIA  \n \n_________________________________________________________________________________  \n \nTÓPICOS  \n1. Lúpus Eritematoso Sistêmico  \n● Introdução  \n● Quadro clínico  \n○ Manifestações sistêmicas  \n○ Manifestações laboratoriais  \n● Diagnóstico  \n● Tratamento'),
 Document(page_content='● Tratamento  \n● Gravidez e contracepção  \n● Síndrome do Anticorpo Antifosfolipídeo (SAF)  \n2. Esclerodermia  \n● Introdução  \n● Classificação  \n○ Forma localizada  \n○ Forma sistêmica  \n■ Cutânea difusa  \n■ Cutânea limitada - CREST  \n■ Visceral  \n● Quadro clínico  \n● Diagnóstico  \n● Tratamento'),
 Document(page_content='● Tratamento  \n3. Síndrome de Sjögren  \n● Introdução  \n● Quadro clínico  \n● Exames laboratoriais  \n \n \nJéssica Nicolau - 2 ● Diagnóstico  \n● Tratamento  \n4. Vasculites  \n● Introdução  \n● Vasculites de grandes vasos  \n○ Arterite Temporal  \n○ Arterite de Takayasu  \n● Vasculites de médios vasos'),
 Doc

In [6]:
loader = PyPDFLoader(file_path)
data = loader.load()
data[:2]

[Document(page_content=' \nJéssica Nicolau - 1 REUMATOLOGIA  \n \n_________________________________________________________________________________  \n \nTÓPICOS  \n1. Lúpus Eritematoso Sistêmico  \n● Introdução  \n● Quadro clínico  \n○ Manifestações sistêmicas  \n○ Manifestações laboratoriais  \n● Diagnóstico  \n● Tratamento  \n● Gravidez e contracepção  \n● Síndrome do Anticorpo Antifosfolipídeo (SAF)  \n2. Esclerodermia  \n● Introdução  \n● Classificação  \n○ Forma localizada  \n○ Forma sistêmica  \n■ Cutânea difusa  \n■ Cutânea limitada - CREST  \n■ Visceral  \n● Quadro clínico  \n● Diagnóstico  \n● Tratamento  \n3. Síndrome de Sjögren  \n● Introdução  \n● Quadro clínico  \n● Exames laboratoriais  \n \n', metadata={'source': '//home/rps/Downloads/REUMATOLOGIA.pdf', 'page': 0}),
 Document(page_content=' \nJéssica Nicolau - 2 ● Diagnóstico  \n● Tratamento  \n4. Vasculites  \n● Introdução  \n● Vasculites de grandes vasos  \n○ Arterite Temporal  \n○ Arterite de Takayasu  \n● Vasculites

In [7]:
# file_path = '//home/rps/Downloads/REUMATOLOGIA.pdf'

# document_ques_gen, document_answer_gen = file_processing(file_path)

# #llm_ques_gen_pipeline = load_llm()

# prompt_template = """
# You are an expert at creating questions based in medicine materials and documentation.
# Your goal is to prepare a very good material for their exam and  tests.
# You do this by asking questions about the text below:

# ------------
# {text}
# ------------

# Create questions that will prepare medicine students for their tests.
# Make sure not to lose any important information.

# QUESTIONS:
# """

# PROMPT_QUESTIONS = PromptTemplate(template=prompt_template, input_variables=["text"])

# refine_template = ("""
# You are an expert at creating practice questions based on medicine and books.
# Your goal is to help to prepare medicine students for a test.
# We have received some practice questions to a certain extent: {existing_answer}.
# We have the option to refine the existing questions or add new ones.
# (only if necessary) with some more context below.
# ------------
# {text}
# ------------

# Given the new context, refine the original questions in Portuguese.
# If the context is not helpful, please provide the original questions.
# QUESTIONS:
# """
# )

# REFINE_PROMPT_QUESTIONS = PromptTemplate(
#     input_variables=["existing_answer", "text"],
#     template=refine_template,
# )

# ques_gen_chain = load_summarize_chain(llm = llm, 
#                                         chain_type = "refine",
#                                         verbose = True, 
#                                         question_prompt=PROMPT_QUESTIONS, 
#                                         refine_prompt=REFINE_PROMPT_QUESTIONS)

# ques_l = [ ques_gen_chain.run( [q]) for q in document_ques_gen[:2]]
# ques = "\n".join(ques_l).split("\n")

In [8]:
# [element for element in ques if element.endswith('?') or element.endswith('.')]

In [9]:
ques, vector_store = pipe_vector(file_path)



/home/rps/miniconda3/envs/llm/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new RefineDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

    You are an expert at creating questions based in medicine materials and documentation.
    Your goal is to prepare a very good material for their exam and  tests.
    You do this by asking questions about the text below:

    ------------
    Jéssica Nicolau - 1 REUMATOLOGIA  
 
_________________________________________________________________________________  
 
TÓPICOS  
1. Lúpus Eritematoso Sistêmico  
● Introdução  
● Quadro clínico  
○ Manifestações sistêmicas  
○ Manifestações laboratoriais  
● Diagnóstico  
● Tratamento  
● Gravidez e contracepção  
● Síndrome do Anticorpo Antifosfolipídeo (SAF)  
2. Esclerodermia  
● Introdução  
● Classificação  
○ Forma localizada  
○ Forma sistêmica  
■ Cutânea difusa  
■ Cutânea limitada - CREST  
■ Visceral  
● Quadro clínico  
● Diagnóstico  
● Tratamento  
3. Síndrome de Sjögren  
● Introdução  
● Quadro clínico  
● Exames labo

In [11]:
len(ques)

2015

In [12]:
answer_generation_chain, ques_list = pipe_gen(vector_store=vector_store, ques= ques)
base_folder = 'static/output/'
if not os.path.isdir(base_folder):
    os.mkdir(base_folder)
output_file = base_folder+"QAxy.csv"
with open(output_file, "w", newline="", encoding="utf-8") as csvfile:
    csv_writer = csv.writer(csvfile)
    csv_writer.writerow(["Question", "Helpful Answer"])  # Writing the header row
    #answer0=[]
    for question in ques_list:
        print("Question: ", question)
        answer = answer_generation_chain.run(question)
        #answer0.append(answer)
        print("Answer: ", answer)
        print("--------------------------------------------------\n\n")

        # Save answer to CSV file
        csv_writer.writerow([question, answer])
# 211 min

Question:  1. What are the topics covered in this material?
Answer:   The material covers concepts related to joint infection, clinical presentation of acute monoarthritis, PAN syndrome, cutaneous vasculitis, renal insufficiency and renovascular HAS, osteophyte formation, affected joints, extra-intestinal manifestations, and treatment guidelines for not prescribing antibiotics.
--------------------------------------------------


Question:      2. What is Lúpus Eritematoso Sistêmico and what are its manifestations?
Answer:    Lúpus Eritematoso Sistêmico (LES) is a chronic autoimmune disease that can affect various parts of the body, including the skin, joints, kidneys, blood cells, heart, and brain. Its manifestations include rash malar (a butterfly-shaped rash across the cheeks and nose), photosensitivity (increased sensitivity to sunlight), lúpus discoide (eritematous and desquamative lesions similar to psoriasis), non-evolving with systemic LES, anti-Ro positive, vasculite lúpica (l

In [16]:
num_documents = len(vector_store.index_to_docstore_id)
print(f"Total number of documents: {num_documents}")

vector_store.save_local("faiss_index")

Total number of documents: 525


In [18]:
query = "como tratar artrite septica?"
docs = vector_store.similarity_search(query)
docs

[Document(page_content='apenas se dúvida quanto a possibilidade de artrite séptica!  \n● RX com achado típico de lesão em saca -bocado ! \n \n  \n \n- Tratamento : \n● Artrite Gotosa Aguda:  \n○ NÃO FAZER AAS E NÃO MEXER NAS DOSES DAS DROGAS HIPOURICEMIANTES \nNESSE MOMENTO, POIS TAIS MEDICAMENTOS VARIAM ABRUPTAMENTE OS'),
 Document(page_content='● Tratar infecção, se presente  \n \n \n \n \nJéssica Nicolau - 54 5) ARTRITE PSORIÁSICA  \n- Será abordada no capítulo de " Espondiloartrites"  \n● Formas de artrite:  \n○ Oligoartrite assimétrica (semelhante à Síndrome de Reiter)  \n○ Poliartrite simétrica (semelhante a AR)'),
 Document(page_content='○ Ciprofloxacino 500 mg 12/12h até completar 7 dias de tratamento  \n○ Lavagem por artroscopia raramente é necessária!  \n● Não Gonocócica:  \n○ Antibioticoterapia:  \n■ Oxacilina  \n■ Ceftriaxone, se dúvida em origem gonocócica  \n○ Punções seriadas esvaziadores , pelo risco de destruição articular'),
 Document(page_content='● Como tratar?  \n○

In [31]:
for question in ques_list:
        print("Question: ", question)
        answer = answer_generation_chain.run(question)
        print("Answer: ", answer)

Question:  1. Qual é a definição do Transtorno de Ansiedade Geralizado (TAG)?
Answer:   The Transtorno de Ansiedade Geralizado (TAG) is characterized by a continuous and exaggerated state of anxiety, worry, and tension that lasts for at least six months.
Question:      2. Quanto da população está afetada pelo TAG?
Answer:   Entre 5% e 10% da população está afetada pelo TAG.

Question:     3. Qual o sexo mais afetado pelo TAG?
Helpful Answer: As mulheres são de duas a três vezes mais afetadas do que o sexo masculino.

Question:     4. Quais os principais tratamentos para o TAG?
Helpful Answer: O melhor tratamento é a combinação de métodos, incluindo psicoterapia cognitivo-comportamental.

Question:     5. Qual a idade em que o TAG geralmente começa?
Helpful Answer: O TAG normalmente se inicia na adolescência.

Question:     6. Quanto da população está afetada pelo transtorno de ansiedade social (TAS)?
Helpful Answer: O TAS acomete cerca 13% da população.

Question:     7. Qual o sexo ma

In [28]:
ques_list

['1. Qual é a definição do Transtorno de Ansiedade Geralizado (TAG)?',
 '    2. Quanto da população está afetada pelo TAG?',
 '    3. Homens ou mulheres são mais propensos a serem afetados pelo TAG?',
 '    4. A que idade o TAG geralmente começa?',
 '    5. Como é tratado o TAG?',
 '    6. Quais medicamentos são considerados como primeira linha de tratamento para o TAG?',
 '    7. Qual é a prognose para pacientes com TAG que não recebem tratamento adequado?.']

In [4]:
file_path = '/home/rps/Downloads/ansiedade_4_8.pdf'

output_file = get_csv(file_path)

/home/rps/miniconda3/envs/llm/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new RefineDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

    You are an expert at creating questions based in medicine materials and documentation.
    Your goal is to prepare a very good material for their exam and  tests.
    You do this by asking questions about the text below:

    ------------
    Estratégia
MEDPSIQUIATRIA Transtorno de Ansiedade
Prof. Thales Thaumaturgo | Resumo Estratégico | 202341.0 TRANSTORNOS DE ANSIEDADE
1.1 TRANSTORNO DE ANSIEDADE GENERALIZADA (TAG)
O transtorno de ansiedade generalizada (TAG)  é caracterizado por um estado contínuo e exagerado de ansiedade, preocupação e 
tensão que dura pelo menos 6 meses. 
1.1.1 EPIDEMIOLOGIA, CURSO E PROGNÓSTICO
O TAG afeta entre 5% e 10% da população e as mulheres são de duas a três vezes mais afetadas do que o sexo masculino. Esse 
transtorno normalmente se inicia na adolescência, tem curso crônico e flutuante, que se mistura com a história de vida do indivíduo. Pouco